# ETL (Extract, Transform, Load)

Este notebook realiza o processo de ETL (Extract, Transform, Load) de dados de pedidos, consumidores e restaurantes, com o objetivo de preparar e analisar dados para experimentos de negócio. As principais etapas incluem:

- **Carregamento de Dados Brutos:** Utiliza funções utilitárias para carregar dados brutos de pedidos, consumidores, restaurantes e mapeamento de IDs.
- **Limpeza e Conformação:** Aplica regras de limpeza, tratamento de timezone e conformação dos dados para garantir consistência e qualidade.
- **Construção de Camadas Silver:** Gera tabelas intermediárias ("silver") de pedidos e usuários, agregando informações relevantes como recência, frequência e valor monetário.
- **Análises Exploratórias:** Realiza inspeções de schema, contagem de linhas, análise de nulos, faixas de datas, splits por grupos experimentais, distribuição de valores e estatísticas descritivas.
- **Validações:** Executa asserts para garantir unicidade de IDs e ausência de nulos críticos.
- **Visualizações:** Gera gráficos para explorar a distribuição dos valores de pedidos e sumariza métricas por grupo experimental.

O notebook serve como base para análises e validações de experimentos, garantindo integridade e qualidade dos dados processados.

In [ ]:
import os, sys, subprocess
from pathlib import Path

GITHUB_USER = "silvaniacorreia"
REPO_NAME   = "ifood-case-cupons"

IN_COLAB = "COLAB_RELEASE_TAG" in os.environ or "COLAB_GPU" in os.environ

if IN_COLAB:
    PROJECT_DIR = Path("/content") / REPO_NAME
    os.chdir(PROJECT_DIR)
else:
    PROJECT_DIR = Path.cwd()

if str(PROJECT_DIR) not in sys.path:
    sys.path.insert(0, str(PROJECT_DIR))

from src.utils import load_settings, get_spark
from src import etl
from pyspark.sql import functions as F

s = load_settings()
spark = get_spark(
    app_name=s.runtime.spark.app_name,
    shuffle_partitions=s.runtime.spark.shuffle_partitions
)

orders, consumers, restaurants, abmap = etl.load_raw(spark, s.data.raw_dir)

win = getattr(s.analysis, "experiment_window", None)
start = win.get("start") if isinstance(win, dict) else None
end   = win.get("end")   if isinstance(win, dict) else None
auto  = getattr(s.analysis, "auto_infer_window", True)

df = etl.clean_and_conform(
    orders, consumers, restaurants, abmap,
    business_tz=getattr(s.analysis, "business_tz", "America/Sao_Paulo"),
    treat_is_target_null_as_control=getattr(s.analysis, "treat_is_target_null_as_control", False),
    experiment_start=start,
    experiment_end=end,
    auto_infer_window=auto,
)

orders_silver = etl.build_orders_silver(df)
orders_silver.write.mode("overwrite").parquet(f"{s.data.processed_dir}/orders_silver.parquet")

users_silver = etl.build_user_aggregates(orders_silver)
ref_ts = orders_silver.agg(F.max("event_ts_utc")).first()[0]
users_silver = users_silver.withColumn("recency", F.datediff(F.lit(ref_ts), F.col("last_order")))
users_silver.write.mode("overwrite").parquet(f"{s.data.processed_dir}/users_silver.parquet")

display(orders_silver.limit(5).toPandas())
display(users_silver.limit(5).toPandas())
print("ref_ts:", ref_ts)
print("ETL concluído com sucesso!")

[ETL] Janela INFERIDA a partir dos dados (UTC): start=2019-01-17 end=2019-01-18 (end exclusivo)
Spark: 3.5.1
Hadoop: 3.3.4


Py4JJavaError: An error occurred while calling o1141.parquet.
: java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://wiki.apache.org/hadoop/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:735)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:270)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:286)
	at org.apache.hadoop.fs.RawLocalFileSystem.setPermission(RawLocalFileSystem.java:978)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkOneDirWithMode(RawLocalFileSystem.java:660)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:700)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:672)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:699)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.

## Checagens dos dados

In [ ]:
# Esquema, contagens e unicidade de chaves

print("=== orders_silver ===")
orders_silver.printSchema()
n_orders = orders_silver.count()
n_orders_distinct = orders_silver.select("order_id").distinct().count()
print("linhas:", n_orders, "| order_id distintos:", n_orders_distinct)

print("\n=== users_silver ===")
users_silver.printSchema()
n_users  = users_silver.count()
n_users_distinct = users_silver.select("customer_id").distinct().count()
print("linhas:", n_users, "| customer_id distintos:", n_users_distinct)


In [ ]:
# Nulos por coluna 

def nulls_by_col(df):
    exprs = [F.sum(F.col(c).isNull().cast("int")).alias(c) for c in df.columns]
    return df.select(exprs)

print("Nulos em orders_silver:")
nulls_by_col(orders_silver).show(truncate=False)

print("Nulos em users_silver:")
nulls_by_col(users_silver).show(truncate=False)


In [ ]:
# Faixa de datas e janela do experimento

print("Faixa de datas (UTC) em orders_silver:")
orders_silver.agg(F.min("event_ts_utc").alias("min_utc"),
                  F.max("event_ts_utc").alias("max_utc")).show()

print("Contagem diária (BRT) por grupo:")
(orders_silver.groupBy("event_date_brt","is_target")
 .count()
 .orderBy("event_date_brt","is_target")
 .show(40))


In [ ]:
# Balanceamento do A/B

print("Split por is_target (orders):")
orders_silver.groupBy("is_target").count().show()

print("Split por is_target (users):")
users_silver.groupBy("is_target").count().show()


In [ ]:
# Valores monetários — sanity check e outliers

print("Negativos/zero em order_total_amount:")
neg = orders_silver.filter(F.col("order_total_amount") < 0).count()
zero = orders_silver.filter(F.col("order_total_amount") == 0).count()
print("negativos:", neg, "| zero:", zero)

print("Resumo de order_total_amount:")
orders_silver.select("order_total_amount").summary().show()

print("Quantis aproximados (1%, 5%, 50%, 95%, 99%):")
q = orders_silver.approxQuantile("order_total_amount", [0.01, 0.05, 0.5, 0.95, 0.99], 0.01)
q


In [ ]:
# Top outliers (visão pontual)

orders_silver.orderBy(F.desc("order_total_amount")).select(
    "order_id","customer_id","order_total_amount","event_ts_utc","is_target"
).show(10, truncate=False)


In [ ]:
#  Distribuições

import matplotlib.pyplot as plt

pdf = (orders_silver
       .select("order_total_amount")
       .sample(withReplacement=False, fraction=0.2, seed=42)  # 20% de amostra
       .toPandas()
       .dropna())

plt.figure()
plt.hist(pdf["order_total_amount"], bins=50)
plt.title("Distribuição de order_total_amount (amostra)")
plt.xlabel("R$")
plt.ylabel("Contagem")
plt.show()


In [ ]:
# Métricas por grupo

preview = (orders_silver.groupBy("is_target")
           .agg(
               F.count("*").alias("n_orders"),
               F.sum("order_total_amount").alias("gmv"),
               F.avg("order_total_amount").alias("avg_ticket"),
           )
          )
preview.show()


In [ ]:
# Sanidade de atributos de apoio

print("language por grupo (users):")
(users_silver.groupBy("is_target","language")
 .count()
 .orderBy("language","is_target")
 .show(20))

print("active rate por grupo (users):")
(users_silver.groupBy("is_target")
 .agg(F.avg(F.col("recency").isNotNull().cast("double")).alias("has_recency"),
      F.avg(F.col("frequency")).alias("avg_freq"),
      F.avg(F.col("monetary")).alias("avg_monetary"))
 .show())


In [ ]:
# Checklist final do ETL

assert n_orders == n_orders_distinct, "order_id duplicado no silver de pedidos"
assert n_users == n_users_distinct,   "customer_id duplicado no silver de usuários"
assert orders_silver.filter(F.col("event_ts_utc").isNull()).count() == 0, "event_ts_utc nulo"
assert users_silver.filter(F.col("is_target").isNull()).count() == 0, "is_target nulo em users"
print("✔️ ETL checks básicos OK.")
